pip install prophet --quiet - встановіть через термінал.
pip install pandas
pip install matplotlib

In [28]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt


Loading of DataSet, Some checks and data prep

In [29]:
# Load dataset
df = pd.read_csv('test_dataset.csv')


In [30]:
# Show first few rows
df.head()

,period,sku_id,category_id,brand,lifetime_days,qty_total,orders_qty,abc,total_abc_numeric,avg_discount_perc_by_goods,...,war,covid,year,sin_quarter,cos_quarter,sin_month,cos_month,sin_week,cos_week,data_status
0,6/1/2024,46303921590,672241,747920275,-44,71,6,B,2,0.08,...,1,0,2024,1.224647e-16,-1.0,1.224647e-16,-1.0,4.647232e-01,-0.885456,inconsistent
1,6/8/2024,46303774477,672242,747920274,143,1,0,B,2,0.26,...,1,1,2024,1.224647e-16,-1.0,1.224647e-16,-1.0,3.546049e-01,-0.935016,inconsistent
2,6/15/2024,46303921590,672242,747920274,24,61,18,C,1,0.49,...,0,0,2024,1.224647e-16,-1.0,1.224647e-16,-1.0,2.393157e-01,-0.970942,inconsistent
3,6/22/2024,46303774477,672242,747920274,114,89,1,C,1,0.34,...,0,1,2024,1.224647e-16,-1.0,1.224647e-16,-1.0,1.205367e-01,-0.992709,inconsistent
4,6/29/2024,46303862950,672242,747920274,-97,3,17,B,2,0.22,...,0,1,2024,1.224647e-16,-1.0,1.224647e-16,-1.0,-3.216245e-16,-1.000000,inconsistent


In [31]:
# Check the column names in your dataset
df.columns

Index(['period', 'sku_id', 'category_id', 'brand', 'lifetime_days',
       'qty_total', 'orders_qty', 'abc', 'total_abc_numeric',
       'avg_discount_perc_by_goods', 'max_discount_perc_by_goods',
       'avg_goods_price_by_goods', 'revenue', 'last_goods_sell_status',
       'oos__by_goods', 'war', 'covid', 'year', 'sin_quarter', 'cos_quarter',
       'sin_month', 'cos_month', 'sin_week', 'cos_week', 'data_status'],
      dtype='object')

In [32]:
#  Convert 'period' to datetime
df['period'] = pd.to_datetime(df['period'])

In [33]:
# Fix numeric columns with commas instead of dots
columns_to_fix = ['avg_discount_perc_by_goods', 'oos__by_goods', 'sin_month', 'cos_month']

for col in columns_to_fix:
    df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [34]:
# Select only required columns (adjust as needed)
df = df[['period', 'sku_id', 'qty_total', 'avg_discount_perc_by_goods', 
         'oos__by_goods', 'sin_month', 'cos_month']]


In [35]:
# Prepare results container
forecast_all = []

In [36]:
# Loop through each unique SKU
sku_list = df['sku_id'].unique()

for sku in sku_list:
    sku_df = df[df['sku_id'] == sku].copy()

    # Skip SKUs with too little data
    if len(sku_df) < 10:
        continue

    # Rename columns to match Prophet requirements
    sku_df.rename(columns={'period': 'ds', 'qty_total': 'y'}, inplace=True)

    # Drop rows with missing values in regressors
    sku_df = sku_df.dropna(subset=['avg_discount_perc_by_goods', 
                                   'oos__by_goods', 'sin_month', 'cos_month'])
 
     # Drop rows with missing regressor values
    sku_df = sku_df.dropna(subset=['avg_discount_perc_by_goods', 
                                   'oos__by_goods', 'sin_month', 'cos_month'])

    # Skip if less than 2 rows after cleaning
    if len(sku_df) < 2:
        continue
    # Initialize Prophet with additional regressors
    model = Prophet()
    model.add_regressor('avg_discount_perc_by_goods')
    model.add_regressor('oos__by_goods')
    model.add_regressor('sin_month')
    model.add_regressor('cos_month')

    model.fit(sku_df)

    # Create future dataframe for 28 days
    future = model.make_future_dataframe(periods=28)

    # Add future regressor values (using last known value)
    for col in ['avg_discount_perc_by_goods', 'oos__by_goods', 'sin_month', 'cos_month']:
        last_val = sku_df[col].iloc[-1]
        future[col] = last_val  # repeat last known value

    # Forecast
    forecast = model.predict(future)


    result = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(28).copy()
    result.rename(columns={
        'ds': 'date',
        'yhat': 'forecast_qty_total',
        'yhat_lower': 'forecast_qty_total_lower',
        'yhat_upper': 'forecast_qty_total_upper'
    }, inplace=True)


    result['sku_id'] = sku


    forecast_all.append(result)

16:11:09 - cmdstanpy - INFO - Chain [1] start processing
16:11:09 - cmdstanpy - INFO - Chain [1] done processing
16:11:09 - cmdstanpy - INFO - Chain [1] start processing
16:11:09 - cmdstanpy - INFO - Chain [1] done processing
16:11:10 - cmdstanpy - INFO - Chain [1] start processing
16:11:10 - cmdstanpy - INFO - Chain [1] done processing


Getting a result, saving to Csv.

In [37]:
#  Combine all forecasts
final_forecast_df = pd.concat(forecast_all, ignore_index=True)


In [38]:

# Save to CSV
final_forecast_df.to_csv('forecast_28_days_all_skus_with_regressors.csv', index=False)

In [39]:

# Show sample of result
final_forecast_df.head()

,date,forecast_qty_total,forecast_qty_total_lower,forecast_qty_total_upper,sku_id
0,2024-08-18,0.624051,-25.419002,28.681609,46303921590
1,2024-08-19,0.026735,-26.944013,28.260282,46303921590
2,2024-08-20,-0.570582,-28.970171,27.973135,46303921590
3,2024-08-21,-1.167898,-28.102775,26.297369,46303921590
4,2024-08-22,-1.765215,-29.388065,26.003015,46303921590
